<a href="https://colab.research.google.com/github/anko191/Python_Kaggle/blob/master/%5BMulti-Label%5DNN/Multi_LabelClassificationwithDeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 始めに
* Multi-label classificationはzeroも入るし、もっと多いクラスのラベルを持ったりする
* クラスラベルがお互いに、排他的である普通の分類とは違って、
    * ﾏﾙﾁラベル分類では、複数の相互に排他的でないクラスでないクラスや「ラベル」
    * をサポートする特殊な機械学習アルゴリズムが必要となる
* Deep Learning neural networksは、マルチラベル分類問題ためのもの
    * Keras libraryを使用して簡単に定義して評価できる



# multilabel分類 - 


* ゼロ以上の相互に排他的でないクラス・ラベルを予測するモデリングタスク。
* NN Modelは、マルチラベル分類タスクのために設定できる
* 以下3つに分かれています
    * Multi-Label Classification
    * Neural Networks for Multiple Labels
    * Neural Network for Multi-Label Classification


## Multi-Label Classification
* 普通は分類タスクでは、単一のラベルを予測
* 別の方法では、2つ以上のクラス・ラベルにわたって尤度を予測する
    * クラスは相互に排他的であり、
    * 分類タスクは入力が1つのクラスにのみ属すると仮定することを意味
* 分類作業の中には、**複数のクラス・ラベルを予測する**必要があります
* これはクラス・ラベルまたは、クラス・メンバーシップが相互に排他的ではないことを意味
* これらのタスクは multi-label classification(multiple label classification)

* sklearnの、**make_multilabel_classification()** を使う。
    * a synthetic multi-label classification dataset を作成
    * 合成マルチラベル分類データセット
* このデータセットは、**10個の入力特徴量**, **1000個のサンプルを持ちます**
    * 各サンプルに対して3つのクラスラベル出力を持ち、
    * 各クラスは1つまたは2つの値(0 or 1, present or not present (存在するしない))

### multi-label classification task

In [1]:
from sklearn.datasets import make_multilabel_classification

### データセットを定義しましょう

In [2]:
X, y = make_multilabel_classification(n_samples = 1000, n_features = 10,
                                      n_classes = 3, n_labels = 2,
                                      random_state = 1)
print(X.shape, y.shape)

(1000, 10) (1000, 3)


In [3]:
for i in range(10):
    print(X[i], y[i])

[ 3.  3.  6.  7.  8.  2. 11. 11.  1.  3.] [1 1 0]
[7. 6. 4. 4. 6. 8. 3. 4. 6. 4.] [0 0 0]
[ 5.  5. 13.  7.  6.  3.  6. 11.  4.  2.] [1 1 0]
[1. 1. 5. 5. 7. 3. 4. 6. 4. 4.] [1 1 1]
[ 4.  2.  3. 13.  7.  2.  4. 12.  1.  7.] [0 1 0]
[ 4.  3.  3.  2.  5.  2.  3.  7.  2. 10.] [0 0 0]
[ 3.  3.  3. 11.  6.  3.  4. 14.  1.  3.] [0 1 0]
[ 2.  1.  7.  8.  4.  5. 10.  4.  6.  6.] [1 1 1]
[ 5.  1.  9.  5.  3.  4. 11.  8.  1.  8.] [1 1 1]
[ 2. 11.  7.  6.  2.  2.  9. 11.  9.  3.] [1 1 1]


* 出力クラス・ラベルが3つのクラス・ラベルのそれぞれについて0または1の値を持っている

## Neural Networks for Multiple Labels
* 出力のノード数として問題に存在するターゲットラベルの数を指定するだけ
* 例えば、3つの出力ラベルを持つタスクでは、
    * 出力層に3つのノードを持つニューラルネットワークの出力層が必要になります
* 出力層の各ノードは、sigmoid activation を利用しなければならない
    * ラベルを0, 1の間の値で予測
* 最後に、モデルは2値cross-entropy loss functionで適合させないといけない。
* 要約すると、
    * 出力層のノードの数はラベルの数と一致する
    * 出力層の各ノードのsigmoid activation
    * Binary cross-entropy loss function
* Keras deeplearning libraryを使用
* 多層パーセプトロン(MLP)モデルを定義します。

* 各サンプルは 10 inputs と 3 outputsの出力を持っています
    * 従って、networkは最初の隠れそうのinput_dim の 10inputsと、
    * 出力層の3つのノードを期待する入力層を必要とします。
* 隠れ層では、一般的な **ReLU activation function**を使います
    * 隠れ層には、試行錯誤の末に選ばれた20個のノードがあります。
    * 我々は、
        * binary cross-entropy loss
        * Adam (確率的勾配降下)

### model を定義

In [4]:
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim = n_inputs, kernel_initializer = 'he_uniform', activation = 'relu'))
    model.add(Dense(n_outputs, activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam')

    return model

## Neural Network for Multi-Label Classification
* MLP(Multilayer Perceptron) for multi-label crassification を定義出来たので、
* どのように評価されるか探ってみましょう

* **データセットが小さい場合は**
    * 同じデータセットでニューラルネットワークモデルを繰り返し評価し、
    * 繰り返しの平均性のを報告するのが良い方法です
* このことを考慮に入れて、我々は、10倍と3回の繰り返しのk-fold交差検証を用いて、多出力回帰タスクでMLPモデルを評価します。
* 以下の evaluate_model() 関数は、データセットを受け取り、モデルを評価し、評価スコアのリスト（この場合は精度スコア）を返します。

## 完成版

In [5]:
# mlp for multi-label classification
from numpy import mean, std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score

In [9]:
# get the dataset
def get_dataset():
    X, y = make_multilabel_classification(n_samples = 1000,
                                          n_features = 10,
                                          n_classes = 3,
                                          n_labels = 2,
                                          random_state = 1,
                                          )
    return X,y

In [10]:
# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim = n_inputs, kernel_initializer='he_uniform', activation = 'relu'))
    model.add(Dense(n_outputs, activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer='adam')
    return model

In [ ]:
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    # define evaluation procedure
    cv = RepeatedKFold(n_splits = 10, n_repeats = 3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        # fit model
        model.fit(X_train, y_train, verbose = 1, epochs = 100)
        # make a prediction on the test set
        pred_y = model.predict(X_test)
        # round probabilities to class labels
        pred_y = pred_y.round()
        # calculate accuracy
        acc = accuracy_score(y_test, pred_y)
        # store result
        print('>%.3f'%acc)
        results.append(acc)
    return results
# load dataset
X,y = get_dataset()
# evaluate model
results = evaluate_model(X, y)
# summarize performance
print('Accuracy: %.3f (%.3f)'%(mean(results), std(results)))

* このコードは、独自のマルチラベル分類タスクでMLPモデルを評価するためのテンプレートとして使用することができます。
* モデル内のノードやレイヤーの数は、簡単に適応させることができ、
* データセットの複雑さに合わせて調整することができます。

* モデル構成が選択されると、それを使用して、利用可能なすべてのデータに最終モデルを適合させ、新しいデータの予測を行うことができます。

* 以下の例は、
    * 最初にマルチラベル分類データセット全体にMLPモデルを適合させ、
    * 次に保存されたモデルでpredict()関数を呼び出して、
    * データの新しい行の予測を行うことで、これを実証しています。

* 例題を実行すると、モデルが適合し、新しい行の予測を行います。
* 予想通り、予測には、マルチラベル分類タスクに必要な3つの出力変数、
    * すなわち、**各クラス・ラベルの確率**が含まれています。

In [13]:
from numpy import asarray
from sklearn.datasets import make_multilabel_classification
from keras.models import Sequential
from keras.layers import Dense

# def get the dataset
def get_dataset():
    X,y = make_multilabel_classification(n_samples = 1000,
                                         n_features = 10,
                                         n_classes = 3,
                                         n_labels = 2,
                                         random_state = 1)
    return X,y
# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim = n_inputs, 
                    kernel_initializer='he_uniform',
                    activation = 'relu'))
    model.add(Dense(n_outputs, activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer='adam')
    return model
# load dataset
X,y = get_dataset()
n_inputs, n_outputs = X.shape[1], y.shape[1]
# get model
model = get_model(n_inputs, n_outputs)
# fit the model on all data
model.fit(X, y, verbose = 0, epochs = 100)
# make a prediction for new data
row = [3,3,6,7,8,2,11,11,1,3]
newX = asarray([row])
print(newX)
pred_y = model.predict(newX)
print('Predicted: %s'%pred_y[0])

[[ 3  3  6  7  8  2 11 11  1  3]]
Predicted: [0.99935555 0.9362115  0.03240624]
